In [4]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.smalltest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel'
model_name = "microsoft/phi-2"
label_column = 'sentence_class'
text_column = 'tweet'
few_shot_num = 8
few_shot_selection = "semantic_similarity"
token_path = "/home/dzhang5/.cache/huggingface/token"
cache=False
console_output=False
temperature=0.1
random_shuffle_examples = True
random_shuffle_examples_seed = 1

In [1]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION 
import sys

In [2]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [3]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [5]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [6]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [ ]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetClassificationQA_{few_shot_selection}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetClassificationQA_{few_shot_selection}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetClassificationQA_{few_shot_selection}_{model_name.split('/')[1]}"

In [7]:
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [8]:
test_data = pd.read_csv(data_path)

In [9]:
task_guideline = ("You are an expert at identifying foodborne illness incident information. For the given text, "
                  "your task is to evaluate the text to determine if it describes a potential foodborne illness event. "
                  '''Please answer with "Yes" if it describes a potential foodborne illness event, otherwise answer with "No".'''
                  "Use the following examples as a guide for your predictions and format your responses similarly.")

In [10]:
config = {
    "task_name": task_name,
    "task_type": "question_answering",
    "dataset": {
        "label_column": label_column,
        "text_column": text_column,
        "delimiter": ","
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": task_guideline,
        "output_guidelines": '''You will answer with just the correct output label ("Yes" or "No") and nothing else.''',
        "labels": [
            "Yes",
            "No"
        ],
        "few_shot_examples": few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Input: {{{text_column}}}\nOutput: {{{label_column}}}"
    }
}

config = NewAutoLabelConfig(config)

In [11]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at microsoft/phi-2 were not used when initializing PhiForCausalLM: ['model.layers.25.self_attn.k_proj.weight', 'model.layers.16.self_attn.k_proj.weight', 'model.layers.19.self_attn.q_proj.bias', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.6.self_attn.k_proj.bias', 'model.layers.11.self_attn.q_proj.weight', 'model.layers.23.self_attn.k_proj.bias', 'model.layers.30.self_attn.q_proj.bias', 'model.layers.17.self_attn.k_proj.bias', 'model.layers.7.self_attn.q_proj.weight', 'model.layers.21.self_attn.k_proj.bias', 'model.layers.18.self_attn.q_proj.bias', 'model.layers.6.self_attn.q_proj.bias', 'model.layers.14.self_attn.v_proj.weight', 'model.layers.30.self_attn.q_proj.weight', 'model.layers.14.self_attn.q_proj.bias', 'model.layers.9.self_attn.k_proj.weight', 'model.layers.23.self_attn.q_proj.weight', 'model.layers.5.self_attn.k_proj.weight', 'model.layers.13.self_attn.k_proj.bias', 'model.layers.20.self

In [12]:
ds = AutolabelDataset(test_data[[text_column, label_column]], config=config)
agent.plan(ds)

2024-01-26 16:05:08 sentence_transformers.SentenceTransformer INFO: Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2024-01-26 16:05:09 sentence_transformers.SentenceTransformer INFO: Use pytorch device: cuda


Batches: 0it [00:00, ?it/s]

Output()

Batches: 0it [00:00, ?it/s]

Batches: 0it [00:00, ?it/s]

...

Batches: 0it [00:00, ?it/s]

┌──────────────────────────┬──────┐
│ Total Estimated Cost     │ $0.0 │
│ Number of Examples       │ 10   │
│ Average cost per example │ $0.0 │
└──────────────────────────┴──────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at identifying foodborne illness incident information. Within the TWEET-FID dataset, the task is 
to evaluate a tweet in the TWEET-FID dataset to determine if it describes a potential foodborne illness event. 
Please answer with 'yes' if it describes a potential foodborne illness event, otherwise answer with 'no'.Use the 
following examples as a guide for your predictions and format your responses similarly.

You will answer with just the correct output label ('yes' or 'no') and nothing else.

Some examples with their output answers are provided below:

Input: @USER As much fun as I can. Woke up with food poisoning or stomach flu. Been bugging me all day #tmi Almost 
done driving for the day
Output: yes

Input: Today was rough. Think I had a touch of food poisoning earlier. Thought I was gonna die. Yuck. Therabreath 
rinse & mints helped though! lol
Output: yes

Input: I cant come into work today because I have food poisoning. I went out with the lads last night and we had a 
cheap curry, we all ended up ...
Output: yes

Input: @USER @USER that's not good, do you know what you ate to get food poisoning?
Output: yes

Input: I think I got food poison from ABW 
[EMOJI_astonished_face][EMOJI_face_with_medical_mask][EMOJI_face_screaming_in_fear]
Output: yes

Input: I think I have food poison poops & vomit [EMOJI_persevering_face] that damn subway
Output: yes

Input: lmao RT @USER I've eaten off the streets in India & Bhutan, nothing, on a cruise ship - food poisoning 
twice! #TNI #YouNeverKnow
Output: yes

Input: Food poisoning in Mandeville  HTTPURL
Output: no

Now I want you to label the following example:
Input: @USER As much fun as I can. Woke up with food poisoning or stomach flu. Been bugging me all day #tmi Almost 
done driving for the day
Output:

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
# now, do the actual labeling
ds = agent.run(ds)

Output()

Batches: 0it [00:00, ?it/s]

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:389: 
UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in 
sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Batches: 0it [00:00, ?it/s]

Batches: 0it [00:00, ?it/s]

Batches: 0it [00:00, ?it/s]

In [ ]:
metrics = ds.eval()

In [ ]:
ds.df.to_csv(output_path, index=False)
ds.df.to_pickle(output_path.replace('.csv', '.pkl'))